In [1]:
import open_clip
import torch
import tqdm
import numpy as np
import os
import json

os.environ['CUDA_VISIBLE_DEVICES'] = '5'

model, _, preprocess = open_clip.create_model_and_transforms('RN50', pretrained='openai')
tokenizer = open_clip.get_tokenizer('RN50')

model = model.cuda().eval()

@torch.no_grad()
def get_text_feature(caption):
    text_features  = model.encode_text(tokenizer([caption]).cuda())
    text_features /= text_features.norm(dim=-1, keepdim=True)
    return text_features.cpu().numpy()[0]

text_feature = get_text_feature('a dog')
print(text_feature, text_feature.shape)

/home/dchenbs/anaconda3/envs/seq2seq-ae/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[-0.00301128 -0.00967288 -0.00107659 ... -0.02397537  0.0082068
  0.01266683] (1024,)


In [5]:

from segmentation_dataset import SA1BDataset, COCODataset, LVISDataset, SeqMaskDataset, V3DetDataset, VisualGenomeDataset

# sa1b_root = '/home/dchenbs/workspace/datasets/sa1b'
# sa1b_dataset = SA1BDataset(sa1b_root=sa1b_root)

coco_root = '/home/dchenbs/workspace/datasets/coco2017'
coco_dataset = COCODataset(coco_root=coco_root, split='val')

lvis_root = '/home/dchenbs/workspace/datasets/lvis'
coco_root = '/home/dchenbs/workspace/datasets/coco2017'
lvis_dataset = LVISDataset(lvis_root=lvis_root, coco_root=coco_root, split='val')

v3det_root = '/home/dchenbs/workspace/datasets/v3det'
v3det_dataset = V3DetDataset(v3det_root=v3det_root, split='val')

visual_genome_root = '/home/dchenbs/workspace/datasets/VisualGenome'
visual_genome_dataset = VisualGenomeDataset(visual_genome_root=visual_genome_root, split='val')

for dataset in [coco_dataset, lvis_dataset, v3det_dataset, visual_genome_dataset]:
    captions = dataset.get_all_captions()
    print(f'>>> {dataset.dataset_name} has {len(captions)} different captions: {captions[:10]}')

    text_features = {}
    for caption in tqdm.tqdm(captions):
        text_features[caption] = get_text_feature(caption)
    np.save(f'text_features/{dataset.dataset_name}_clip_rn50.npy', text_features)

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!
>>> coco has 80 different captions: ['An image of a person', 'An image of a bicycle', 'An image of a car', 'An image of a motorcycle', 'An image of a airplane', 'An image of a bus', 'An image of a train', 'An image of a truck', 'An image of a boat', 'An image of a traffic light']


100%|██████████| 80/80 [00:00<00:00, 231.99it/s]


>>> lvis has 1203 different captions: ['An image of a aerosol can, spray can, a dispenser that holds a substance under pressure', 'An image of a air conditioner, a machine that keeps air cool and dry', 'An image of a airplane, aeroplane, an aircraft that has a fixed wing and is powered by propellers or jets', 'An image of a alarm clock, a clock that wakes a sleeper at some preset time', 'An image of a alcohol, alcoholic beverage, a liquor or brew containing alcohol as the active agent', 'An image of a alligator, gator, amphibious reptiles related to crocodiles but with shorter broader snouts', 'An image of a almond, oval-shaped edible seed of the almond tree', 'An image of a ambulance, a vehicle that takes people to and from hospitals', 'An image of a amplifier, electronic equipment that increases strength of signals', 'An image of a anklet, ankle bracelet, an ornament worn around the ankle']


100%|██████████| 1203/1203 [00:05<00:00, 232.09it/s]


>>> v3det has 13204 different captions: ["An image of a ashtray, A receptacle for the ash from smokers' cigars or cigarettes.", 'An image of a cash machine, An unattended machine (outside some banks) that dispenses money when a personal coded card is used.', 'An image of a popper, A container for cooking popcorn.', 'An image of a compass, Navigational instrument for finding directions.', 'An image of a rubber band, A narrow band of elastic rubber used to hold things (such as papers) together.', 'An image of a spring, A metal elastic device that returns to its shape or position when pushed or pulled or pressed.', 'An image of a refrigerator magnet, Magnet, which is used to post items.', 'An image of a concrete mixer, A machine with a large revolving drum in which cement is mixed with other materials to make concrete.', 'An image of a crane, Lifts and moves heavy objects; lifting tackle is suspended from a pivoted boom that rotates around a vertical axis.', 'An image of a generator, An e

100%|██████████| 13204/13204 [00:59<00:00, 221.37it/s]


>>> visual_genome has 410472 different captions: ['An image of flowered, table cloth. Artifact made by weaving or felting or knitting or crocheting natural or synthetic fibers.', 'An image of roof edge.', 'An image of red, silver collar. A band that fits around the neck and is usually folded over.', 'An image of white, metal truck. An automotive vehicle suitable for hauling.', 'An image of Canon camera. Equipment for taking photographs (usually consisting of a lightproof box with a lens at one end and light-sensitive film at the other).', 'An image of red red backpack. A bag carried by a strap on your back or shoulder.', 'An image of      HANDLED, large, tongs, set silverware. Tableware made of silver or silver plate or pewter or stainless steel.', 'An image of piled up snow.', 'An image of tan box. A (usually rectangular) container; may have a lid.', 'An image of  white  white fur. The dressed hairy coat of a mammal.']


100%|██████████| 410472/410472 [29:51<00:00, 229.16it/s]
